# Challenge Task 11: Cymbal Retail AI-Ready Analytics Foundation

## Scenario
As part of the **Lumiki Holiday Campaign**, Cymbal Retail is launching a new "Smart Home" product line. The core catalog data in `rscw_oltp_stg_ds` is functional but lacks marketing depth.

## Your Mission
Use BigQuery's native Generative AI capabilities to enrich the product catalog. You will generate synthetic descriptions and reviews, then build a structured extraction process to identify keywords and sentiment.

## Objectives
1. **Initialize AI Infrastructure:** Setup connections and authorize permissions.
2. **Synthetic Content Generation:** Generate data using `AI.GENERATE`.
3. **Relational Transformation:** Flatten JSON reviews into a row-based table.
4. **Structured Insights:** Use `AI.GENERATE_TABLE` to extract keywords and sentiment.

## 1. Environment Setup
### 1.1 Set your variables

In [ ]:
PROJECT_ID_LIST=!gcloud config list --format "value(core.project)" 2>/dev/null
PROJECT_ID=PROJECT_ID_LIST[0]
LOCATION="us-central1"
SOURCE_DS="rscw_oltp_stg_ds"
TARGET_DS="cymbal_retail_ai_ds"

print(f"Project ID: {PROJECT_ID}")
print(f"Target Dataset: {TARGET_DS}")

### 1.2 Setup Dataset and Connection
**TODO:** Create the destination dataset for Cymbal Retail and the Cloud Resource connection.

In [ ]:
%%bigquery --project {PROJECT_ID}
CREATE SCHEMA IF NOT EXISTS `cymbal_retail_ai_ds`
OPTIONS (location = 'us-central1');

In [ ]:
!bq mk --connection --location=us-central1 --connection_type=CLOUD_RESOURCE cymbal_cloud_resource_connection_usc

### 1.3 Authorize Connection Service Account
Run the following cells to grant the connection permission to call Vertex AI.

In [ ]:
# Step 1: Extract the Service Account ID
CONNECTION_PATH = f"{PROJECT_ID}.us-central1.cymbal_cloud_resource_connection_usc"
DESC_CONN = !bq show --format=prettyjson --connection {CONNECTION_PATH}
import json
CONN_DATA = json.loads("".join(DESC_CONN))
SA_EMAIL = CONN_DATA['cloudResource']['serviceAccountId']

print(f"Authorizing Service Account: {SA_EMAIL}")

In [ ]:
# Step 2: Apply IAM Policy Bindings
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SA_EMAIL} --role='roles/bigquery.connectionUser' --format=none --condition=None
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SA_EMAIL} --role='roles/aiplatform.user' --format=none --condition=None

import time
print("Waiting for IAM propagation (60s)...")
time.sleep(60)

## 2. SQL Challenges

### 2.1 Task: Enrich Catalog with AI.GENERATE
**TODO:** Create `cymbal_product_augmented`. Use `AI.GENERATE` to create descriptions and 10 synthetic reviews (JSON) for the 'home_appliances' category.

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `cymbal_retail_ai_ds.cymbal_product_augmented` AS
SELECT
  product_id,
  product_nm,
  AI.GENERATE(
    'cymbal_cloud_resource_connection_usc',
    CONCAT('Generate a compelling product description for: ', product_nm)
  ) AS product_description,
  AI.GENERATE(
    'cymbal_cloud_resource_connection_usc',
    CONCAT('Generate exactly 10 customer reviews in JSON array format for the product: ', product_nm,
           '. Each review should have "rating" (1-5 integer) and "comment" (string) fields. ',
           'Return ONLY the JSON array, no other text.')
  ) AS product_reviews,
  '' AS product_image_gcs_uri,
  '' AS product_user_manual_gcs_uri
FROM
  `rscw_oltp_stg_ds.product_master`
WHERE
  category_nm = 'home_appliances'
AND
  product_id in ('c2f2d10f8b047f5cffef731479a34432' ,'e9aa3fd3f316b44c85f671552a04fa4a' ,'d418ddef3e7017381084956220b00b9d' ,'e403a54af54730eb4f3e8a346d943579' ,'ba9a847a2c023953743ba9e5ef6c88c8' ,'832137af5164ffb89cf407d686aa2d0d' ,'900d210e9ea6ee40dd0290df76bde9fc' ,'520ac02e0e8cdf179eed22f204e3f26d' ,'aa15d9042b3749f119a734d7df2b68bc' ,'ccb125d39320e9d900107ce4109589dd')
LIMIT 10;

In [ ]:
%%bigquery --project {PROJECT_ID}
-- Validate augmented catalog
SELECT product_id, product_nm, product_description, product_reviews
FROM `cymbal_retail_ai_ds.cymbal_product_augmented` LIMIT 5;

#### Task Validation
> **Note:** Once you have successfully generated the augmented product data, go back to the lab guide page and click **Check my progress** for **AT ID: 7131 Enrich Catalog with AI.GENERATE**.

### 2.2 Task: Create Remote Model
**TODO:** Create the remote model `cymbal_gemini_flash` to handle structured table generation.

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE MODEL `cymbal_retail_ai_ds.cymbal_gemini_flash`
REMOTE WITH CONNECTION `us-central1.cymbal_cloud_resource_connection_usc`
OPTIONS (ENDPOINT = 'gemini-2.5-flash');

### 2.3 Task: Flatten JSON Reviews
**TODO:** Before extracting insights, transform the nested JSON reviews in `cymbal_product_augmented` into a relational format.

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `cymbal_retail_ai_ds.cymbal_product_reviews` AS
SELECT
  product_id,
  product_nm,
  CAST(JSON_EXTRACT_SCALAR(review, '$.rating') AS INT64) AS review_rating,
  JSON_EXTRACT_SCALAR(review, '$.comment') AS review_comment
FROM
  `cymbal_retail_ai_ds.cymbal_product_augmented`,
  UNNEST(JSON_EXTRACT_ARRAY(product_reviews)) AS review;

### 2.4 Task: Extract Insights with AI.GENERATE_TABLE
**TODO:** Use `AI.GENERATE_TABLE` and your remote model to extract keywords and sentiment from the flattened reviews into `cymbal_review_insights`.

In [ ]:
%%bigquery --project {PROJECT_ID}

CREATE OR REPLACE TABLE `cymbal_retail_ai_ds.cymbal_review_insights` AS
SELECT * FROM AI.GENERATE_TABLE(
  'cymbal_retail_ai_ds.cymbal_gemini_flash',
  'Extract keywords and sentiment from the following product reviews',
  STRUCT('ARRAY<STRING>' AS keywords, 'STRING' AS sentiment),
  (SELECT product_id, product_nm, review_rating, review_comment FROM `cymbal_retail_ai_ds.cymbal_product_reviews`)
);

#### Task Validation
> **Note:** Once you have successfully extracted the structured keywords and sentiment, go back to the lab guide page and click **Check my progress** for **AT ID: 7132 Extract Insights with AI.GENERATE_TABLE**.

In [ ]:
%%bigquery --project {PROJECT_ID}
-- Validate review insights
SELECT * FROM `cymbal_retail_ai_ds.cymbal_review_insights` LIMIT 10;